In [2]:
#import os
#import ipdb
import numpy as np
import matplotlib.pyplot as plt
from sklearn import datasets
from sklearn.metrics import accuracy_score, mean_squared_error

class NeuralNetwork:
    '''simple feedforward neural network class'''       
    def __init__(self,num_inputs,num_outputs,num_neurons,tf_functions,epochs=100,learning_rate=0.1):
        self.learning_rate = learning_rate
        self.epochs = epochs
        self.num_inputs = num_inputs
        self.num_outputs = num_outputs
        self.num_neurons = num_neurons

        self.min_weight_value = -1
        self.max_weight_value = 1
        self.tf_functions = []
        #self.tf_functions_derivatives = []

        for i in range(0,len(tf_functions)):
            if tf_functions[i] == "logistic":
                self.tf_functions.append(self.tf_logistic)
                #self.tf_functions_derivatives.append(self.tf_logistic_derivative)                
            elif tf_functions[i] == "linear":
                self.tf_functions.append(self.tf_linear)
                #self.tf_functions_derivatives.append(self.tf_linear_derivative)
            else:
                print("Unknown transfer function: %s" %(tf_function[i]))
                exit()
        self.outputs = None # will be updated after calculation is called

        #********************************
        # Initialize weights and bias
        #********************************
        weights_per_layer = []
        biases_per_layer = []
        wsums_per_layer = []  # for helping later activation function approximate derivative calculation
        for l in range(0,len(num_neurons)): # for all layers
            if (l == 0):
                previous_layer_size = len(inputs[0])
            else:
                previous_layer_size = num_neurons[l-1]
            layer_size = num_neurons[l]
            layer_weights = np.zeros((layer_size,previous_layer_size))
            weights_per_layer.append(layer_weights)
            biases_per_layer.append(np.zeros(layer_size))
            wsums_per_layer.append(np.zeros(layer_size))

        self.weights_per_layer = weights_per_layer
        self.biases_per_layer = biases_per_layer    
        self.wsums_per_layer = wsums_per_layer

        self.randomize_weights()
        self.randomize_biases()


    def set_last_layer_weights(self,weights):
        self.weights_per_layer[-1] = [weights]

    def randomize_weights(self):
        a = self.min_weight_value
        b = self.max_weight_value
        for i, layer in enumerate(self.weights_per_layer):
            #self.weights_per_layer[i] = np.random.random(self.weights_per_layer[i].shape)
            self.weights_per_layer[i] = a+(b-a)*np.random.random(self.weights_per_layer[i].shape)

    def randomize_biases(self):
        a = self.min_weight_value
        b = self.max_weight_value
        for i, layer in enumerate(self.biases_per_layer):
            self.biases_per_layer[i] = a+(b-a)*np.random.random(self.biases_per_layer[i].shape)

    def predict(self,X):
        Ypredicted = []
        for i in range(0,X.shape[0]):  #for every pattern 
            predicted = model.calc_output(X[i]) # calculating outputs for a given pattern
            Ypredicted.append(predicted)
        return np.array(Ypredicted)

    def calc_output(self,inputs):
        '''calculates the output of the neural network'''
        #*****************
        # creates empty output array
        #*****************
        outputs = []
        for i in range(0,len(self.num_neurons)):
            outputs.append(np.zeros(num_neurons[i])) 

        outputs = np.array(outputs) #convert to numpy array
        #*****************
        # calculates output for each layer
        #*****************
        for i in range(0,len(self.num_neurons)):
            if (i == 0): # first layer
                outputs[i] = self.calc_layer(inputs,self.weights_per_layer[i],self.biases_per_layer[i],self.tf_functions[i],i)
            else:
                outputs[i] = self.calc_layer(outputs[i-1],self.weights_per_layer[i],self.biases_per_layer[i],self.tf_functions[i],i)
        self.outputs = outputs
        return outputs[-1]

    def calc_layer(self,inputs,weights,biases,tf_function,layer_index):
        outputs = np.zeros(len(weights))
        for i in range(0,len(weights)): #for all neurons
            outputs[i] = self.neuron(weights[i],biases[i],inputs,tf_function,layer_index,i)
        return outputs

    def tf_logistic(self,x):
        return 1/(1+np.exp(-x*2))
        #return 1/(1+np.exp(-x))

    def tf_logistic_derivative(self,y):        
        return (1-y)*(y) 

    def tf_linear(self,x):
        return x
    def tf_linear_derivative(self,y):
        return 1

    def neuron(self,weights,bias,inputs,tf_function,layer_index,neuron_index):
        weighted_sum = np.dot(weights,inputs)+bias
        output = tf_function(weighted_sum)

        # update for later calculate activation function approximate derivative
        self.wsums_per_layer[layer_index][neuron_index] = weighted_sum

        return output
    def fit(self,X,Y):

        # sequential approach
        self.ssr_total_list = []
        self.mse_total_list = []
        for epc in range(0,self.epochs+1):
            ssr_total = 0
            mse_total = 0
            idxlist = np.arange(0,X.shape[0])
            #on first evaluation, local gradients are not updated, just to save the initial solution            
            np.random.shuffle(idxlist)  #randomize index_list            
            for i in range(0,X.shape[0]):  
                predicted = model.calc_output(X[idxlist[i]]) # calculating outputs for a given pattern  
                output_error = Y[idxlist[i]] - predicted  # error for each output
                ssr = 0.5*sum(output_error**2) #sum of squared residuals
                ssr_total += ssr
                mse_total += sum(output_error**2)
                # calculate local gradients
                if (epc != 0):
                    self.calculate_local_gradients(output_error,ssr)
                    self.update_weights(X[idxlist[i]])
            mse = mse_total/X.shape[0]
            self.ssr_total_list.append(ssr_total)
            self.mse_total_list.append(mse)
            #print("Epoch: %d \t SSR_total = %f" %(epc,ssr_total))



    def calculate_local_gradients(self,output_error,ssr):
        ''' calculates local gradients '''
        # initializes local gradients
        self.local_gradients = [0]*len(num_neurons) # will be a list for each element
        for i in range(0,len(self.local_gradients)):
            self.local_gradients[i] = np.zeros(num_neurons[i])

        # calculates local gradients for output layer
        for j in range(0,self.num_neurons[-1]): # for all neurons in the output layer
            partiald_E_y = -(output_error[j])
            #local_gradient = -partiald_E_y * self.tf_functions_derivatives[-1](self.outputs[-1][j])
            tf = self.tf_functions[-1]
            local_gradient = -partiald_E_y * self.num_derivative(tf,self.wsums_per_layer[-1][j])
            self.local_gradients[-1][j] = local_gradient

        # calculates local gradients for hidden layers
        for l in range(len(num_neurons)-2,-1,-1):  # for all hidden layers, from last to first
            for j in range(0,num_neurons[l]):
                outsum = 0
                w_from_this_neuron_to_next_layer = self.weights_per_layer[l+1][:,j]
                for o in range(0,num_neurons[l+1]): # for all neurons on the next layer
                    wok = w_from_this_neuron_to_next_layer[o]
                    outsum+= self.local_gradients[l+1][o]*wok                
                #self.local_gradients[l][j] = self.tf_functions_derivatives[l](self.outputs[l][j])*outsum
                tf = self.tf_functions[l]
                self.local_gradients[l][j] = self.num_derivative(tf,self.wsums_per_layer[l][j])*outsum

        return 1
    def update_weights(self,inputs):
        '''update weights and bias for backpropagation'''
        # weight update
        for l in range(0,len(num_neurons)): # for all layers
            for j in range(0,num_neurons[l]): # for all neurons in layer
                if (l == 0): # first hidden layer
                    for p in range(0,self.num_inputs): # for all inputs
                        self.weights_per_layer[l][j,p] += self.learning_rate * self.local_gradients[l][j] * inputs[p]
                else:
                    for p in range(0,num_neurons[l-1]): # for all neurons in previous layer
                        self.weights_per_layer[l][j,p] += self.learning_rate * self.local_gradients[l][j] * self.outputs[l-1][p]

                # bias update
                self.biases_per_layer[l][j] += self.learning_rate * self.local_gradients[l][j] * 1
        
        print("Biases per layer: ", self.biases_per_layer)
        print("Pesos por layer: ", self.weights_per_layer)
        print("Local Gradients: ", self.local_gradients)
        
    def num_derivative(self,f,x,delta=1e-6):
        return (f(x+delta)-f(x))/delta


In [3]:
inputs = np.array([[0,0], [0,1], [1,0], [1,1]])
outputs = np.array([0,1,1,0])

num_outputs = np.size(outputs[0]) ## get the number of outputs of the neural network
num_inputs = np.size(inputs[0])

num_neurons = np.array([5, num_outputs])
num_layers = len(num_neurons)

np.random.seed(0)
model = NeuralNetwork(num_inputs, num_outputs, num_neurons, ["logistic", "linear"], epochs = 2000, learning_rate = 0.1)

model.randomize_weights()
model.randomize_biases()

model.fit(inputs,outputs)

model.calc_output(inputs[1])

for i in range(0, len(inputs)):
    print("real output: %f \t predicted output: %f" %(outputs[i], model.calc_output(inputs[i])))

real output: 0.000000 	 predicted output: 0.000000
real output: 1.000000 	 predicted output: 1.000000
real output: 1.000000 	 predicted output: 1.000000
real output: 0.000000 	 predicted output: 0.000000


In [47]:
inputs = np.array([[0,0], [0,1], [1,0], [1,1]])
outputs = np.array([0,1,1,0])

#dataset = dataset.load_iris()
#inputs = dataset.data
#outputs = dataset.target

num_outputs = np.size(outputs[0]) ## get the number of outputs of the neural network
num_inputs = np.size(inputs[0])

num_neurons = np.array([3, num_outputs])
num_layers = len(num_neurons)

learning_rate = 0.1

print(inputs.shape)

np.random.seed(0)
model = NeuralNetwork(num_inputs, num_outputs, num_neurons, ["logistic", "linear"], epochs = 2000, learning_rate = learning_rate)
model.fit(inputs,outputs)
predicted = model.predict(inputs)
predicted_class = np.round(predicted)
accuracy = accuracy_score(outputs,predicted_class)
mse = mean_squared_error(outputs, predicted)

print("MSE: %g \t accuracy: %g" %(mse,accuracy))

(4, 2)
MSE: 1.78803e-30 	 accuracy: 1


In [90]:
#inputs = np.array([[0,0], [0,1], [1,0], [1,1]])
#outputs = np.array([0,1,1,0])

from sklearn.datasets import load_iris

dataset = load_iris()
inputs = dataset.data
outputs_original = dataset.target
print(inputs.shape)
print(outputs_original)

outputs = []
for out_num in outputs_original:
    outlist = [0, 0, 0]
    outlist[out_num] = 1
    outputs.append(outlist)

outputs = np.array(outputs)
print(outputs)
for i in range(0, inputs.shape[1]):
    inputs[:,i] = (inputs[:,i]-min(inputs[:,i]))/(max(inputs[:,i])-min(inputs[:,i]))

num_outputs = np.size(outputs[0]) ## get the number of outputs of the neural network
num_inputs = np.size(inputs[0])
print(num_inputs)
num_neurons = np.array([7, num_outputs])
print(num_neurons)
num_layers = len(num_neurons)
print(num_layers)


learning_rate = 0.01

np.random.seed(0)
model = NeuralNetwork(num_inputs, num_outputs, num_neurons, ["logistic", "linear"], epochs = 600, learning_rate = learning_rate)
model.fit(inputs,outputs)
predicted = model.predict(inputs)
predicted_class = np.round(predicted)
accuracy = accuracy_score(outputs,predicted_class)
mse = mean_squared_error(outputs, predicted)

print("MSE: %g \t accuracy: %g" %(mse,accuracy))

(150, 4)
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2]
[[1 0 0]
 [1 0 0]
 [1 0 0]
 [1 0 0]
 [1 0 0]
 [1 0 0]
 [1 0 0]
 [1 0 0]
 [1 0 0]
 [1 0 0]
 [1 0 0]
 [1 0 0]
 [1 0 0]
 [1 0 0]
 [1 0 0]
 [1 0 0]
 [1 0 0]
 [1 0 0]
 [1 0 0]
 [1 0 0]
 [1 0 0]
 [1 0 0]
 [1 0 0]
 [1 0 0]
 [1 0 0]
 [1 0 0]
 [1 0 0]
 [1 0 0]
 [1 0 0]
 [1 0 0]
 [1 0 0]
 [1 0 0]
 [1 0 0]
 [1 0 0]
 [1 0 0]
 [1 0 0]
 [1 0 0]
 [1 0 0]
 [1 0 0]
 [1 0 0]
 [1 0 0]
 [1 0 0]
 [1 0 0]
 [1 0 0]
 [1 0 0]
 [1 0 0]
 [1 0 0]
 [1 0 0]
 [1 0 0]
 [1 0 0]
 [0 1 0]
 [0 1 0]
 [0 1 0]
 [0 1 0]
 [0 1 0]
 [0 1 0]
 [0 1 0]
 [0 1 0]
 [0 1 0]
 [0 1 0]
 [0 1 0]
 [0 1 0]
 [0 1 0]
 [0 1 0]
 [0 1 0]
 [0 1 0]
 [0 1 0]
 [0 1 0]
 [0 1 0]
 [0 1 0]
 [0 1 0]
 [0 1 0]
 [0 1 0]
 [0 1 0]
 [0 1 0]
 [0 1 0]
 

In [1]:
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import cross_val_score

data = load_boston()

target = data['target']
data = data['data']

target = target.reshape(len(target), 1)

num_outputs = 1
num_inputs = np.size(data[0])
num_neurons = np.array([num_inputs, 1])
num_layers = len(num_neurons)

scaler = MinMaxScaler()
scaler.fit(data)
inputs = scaler.transform(data)
scaler.fit(target)
target = scaler.transform(target)

#target = target.reshape(1, len(target))

train, test, train_labels, test_labels = train_test_split(inputs, target, test_size = 0.2, random_state=42)

learning_rate = 0.1

#print(inputs.shape)
np.random.seed(0)

## Otimizando parametros com todo o conjunto de dados, otimizaremos o learning rate
learning_rate_param = np.arange(0.01, 1, 0.04) # => 25x

scores = []
for i in learning_rate_param:
    model = NeuralNetwork(num_inputs, num_outputs, num_neurons, ["logistic", "linear"], epochs = 100, learning_rate = i)
    model.fit(train,train_labels)
    predicted = model.predict(train)
    predicted_class = np.round(predicted)
    mse = mean_squared_error(train_labels, predicted)
    scores.append(mse)
    print(mse)

scores = np.array(scores)


    

## Lógica do crossvalidation


NameError: name 'np' is not defined

In [33]:
idx = np.argmin(scores)
min_error = min(scores)
learning_rate = 0.01 + idx*0.04

print("Indice de menor erro foi: ", i)
print("Menor erro encontrado no conjunto treinamento: ", min_error)
print("Learning_rate adequado foi: ", learning_rate)


Indice de menor erro foi:  10
Menor erro encontrado no conjunto treinamento:  0.004647158838610101
Learning_rate adequado foi:  0.41000000000000003


In [72]:
##CROSS VALIDATION

train1, test1, train_labels1, test_labels1 = train_test_split(train, train_labels, test_size = 0.33, random_state=42)
train2, test2, train_labels2, test_labels2 = train_test_split(train1, train_labels1, test_size = 0.5, random_state=42)

def cross_validation(x, y, x1, y1, x2, y2, model):
    treino = np.concatenate((x, x1))
    resultado = np.concatenate((y, y1))
    model.fit(treino, resultado)
    predicted = final_model.predict(x2)
    mse = mean_squared_error(y2, predicted)
    return mse
        
model = NeuralNetwork(num_inputs, num_outputs, num_neurons, ["logistic", "linear"], epochs = 100, learning_rate = learning_rate)
a = cross_validation(test1, test_labels1, train2, train_labels2, test2, test_labels2, model) # treinamento = tst1 + train2
b = cross_validation(test1, test_labels1, test2, test_labels2, train2, train_labels2, model) # treinamento = tst1 + tst3
c = cross_validation(test2, test_labels2, train2, train_labels2, test1, test_labels1, model) # treinamneto = train2 + tst3
d = (a+b+c)/3

print("Score com o cross-validation:", d)


Score com o cross-validation: 0.006247132969623345


In [74]:
scorex = []
for i in learning_rate_param:
    model = NeuralNetwork(num_inputs, num_outputs, num_neurons, ["logistic", "linear"], epochs = 100, learning_rate = i)
    a = cross_validation(test1, test_labels1, train2, train_labels2, test2, test_labels2, model) # treinamento = tst1 + train2
    b = cross_validation(test1, test_labels1, test2, test_labels2, train2, train_labels2, model) # treinamento = tst1 + tst3
    c = cross_validation(test2, test_labels2, train2, train_labels2, test1, test_labels1, model) # treinamneto = train2 + tst3
    d = a+b+c
    scorex.append(d)

scorex = np.array(scorex)

In [76]:
idx = np.argmin(scorex)
min_errorr = min(scorex)
learning_ratee = 0.01 + idx*0.04

print("Indice de menor erro foi: ", idx)
print("Menor erro encontrado no conjunto treinamento: ", min_errorr)
print("Learning_rate adequado foi: ", learning_ratee)

Indice de menor erro foi:  6
Menor erro encontrado no conjunto treinamento:  0.015815387804687456
Learning_rate adequado foi:  0.25


In [78]:
model = NeuralNetwork(num_inputs, num_outputs, num_neurons, ["logistic", "linear"], epochs = 100, learning_rate = 0.25)
model.fit(train,train_labels)
predicted = model.predict(test)
mse = mean_squared_error(test_labels, predicted)
print("Score do regressor com o conjunto teste foi: ", mse)

Score do regressor com o conjunto teste foi:  0.008143038949437707


In [101]:
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import cross_val_score

def cross_validation_class(x, y, x1, y1, x2, y2, model):
    treino = np.concatenate((x, x1))
    resultado = np.concatenate((y, y1))
    model.fit(treino, resultado)
    predicted = final_model.predict(x2)
    predicted_class = np.round(predicted)
    accuracy = accuracy_score(y2,predicted_class)
    return accuracy

data = load_breast_cancer()

target = data['target']
data = data['data']

outputs = []
for out_num in target:
    outlist = [0, 0]
    outlist[out_num] = 1
    outputs.append(outlist)

outputs = np.array(outputs)

num_outputs = np.size(outputs[0])
num_inputs = np.size(data[0])
num_neurons = np.array([num_inputs, num_outputs])
num_layers = len(num_neurons)

scaler = MinMaxScaler()
scaler.fit(data)
inputs = scaler.transform(data)

#target = target.reshape(1, len(target))
learning_rate_param = np.arange(0.01, 1, 0.04)

train, test, train_labels, test_labels = train_test_split(inputs, outputs, test_size = 0.2, random_state=42)
train1, test1, train_labels1, test_labels1 = train_test_split(train, train_labels, test_size = 0.33, random_state=42)
train2, test2, train_labels2, test_labels2 = train_test_split(train1, train_labels1, test_size = 0.5, random_state=42)

#print(inputs.shape)
np.random.seed(0)

## Otimizando parametros com todo o conjunto de dados, otimizaremos o learning rate
scorexx = []
for i in learning_rate_param: 
    model = NeuralNetwork(num_inputs, num_outputs, num_neurons, ["logistic", "linear"], epochs = 100, learning_rate = i)
    a = cross_validation_class(test1, test_labels1, train2, train_labels2, test2, test_labels2, model) # treinamento = tst1 + train2
    b = cross_validation_class(test1, test_labels1, test2, test_labels2, train2, train_labels2, model) # treinamento = tst1 + tst3
    c = cross_validation_class(test2, test_labels2, train2, train_labels2, test1, test_labels1, model) # treinamneto = train2 + tst3
    d = (a+b+c)/3
    print(d)
    scorexx.append(d)

scorexx = np.array(scorexx)



0.967047170907401
0.9692401533635412
0.9802195887068664
0.9692401533635412
0.9801905425816196
0.9626612059951202
0.9779975601254792
0.9714186127570582
0.9779975601254792
0.9758045776693388
0.9736115952131986
0.9779975601254792
0.9758045776693388
0.9670326478447775


C:\Users\matheus_sivelli\Anaconda3\lib\site-packages\ipykernel_launcher.py:110: RuntimeWarning: overflow encountered in exp


0.8508191007319623
0.9758045776693388
0.9736115952131986
0.9670326478447775
0.9692256303009179
0.9714186127570582
0.43345532705937034
0.9758045776693388
0.9670326478447775
0.9648396653886372


ValueError: Classification metrics can't handle a mix of multilabel-indicator and multiclass-multioutput targets

In [104]:
idx = np.argmax(scorexx)
min_errorr = min(scorexx)
learning_ratee = 0.01 + idx*0.04

print("Indice de menor erro foi: ", idx)
print("Menor erro encontrado no conjunto treinamento: ", min_errorr)
print("Learning_rate adequado foi: ", learning_ratee)

Indice de menor erro foi:  2
Menor erro encontrado no conjunto treinamento:  0.43345532705937034
Learning_rate adequado foi:  0.09


In [105]:
model = NeuralNetwork(num_inputs, num_outputs, num_neurons, ["logistic", "linear"], epochs = 100, learning_rate = 0.09)
model.fit(train, train_labels)
predicted = final_model.predict(test)
predicted_class = np.round(predicted)
accuracy = accuracy_score(test_labels,predicted_class)
print("Acurácia do modelo com conjunto de teste: ", accuracy)

Acurácia do modelo com conjunto de teste:  0.9824561403508771


In [21]:
inputs = np.array([[0,1]])
outputs = np.array([1])

num_outputs = np.size(outputs[0]) ## get the number of outputs of the neural network
num_inputs = np.size(inputs[0])

num_neurons = np.array([2, num_outputs])
num_layers = 1

np.random.seed(0)
model = NeuralNetwork(num_inputs, num_outputs, num_neurons, ["logistic", "linear"], epochs = 1, learning_rate = 0.1)

model.fit(inputs,outputs)

model.calc_output(inputs[0])

[array([-0.12429063,  0.78297923]), array([0.91964371])]
[array([[0.09762701, 0.43091368],
       [0.20552675, 0.08919959]]), array([[-0.1576697 ,  0.28524696]])]
[array([ 0.00534951, -0.00566771]), array([-0.07681808])]


array([1.06017993])